#### Import of the Packages

In [1]:
import torch
import torchvision
import numpy as np
import math
from torch.utils.data import Dataset,DataLoader
import session_info

session_info.show()

#### Creation of a Dataset

In [2]:
url='https://gist.githubusercontent.com/tijptjik/9408623/raw/b237fa5848349a14a14e5d4107dc7897c21951f5/wine.csv'
xy=np.loadtxt(url, delimiter=',', dtype='float32', skiprows=1)

In [3]:
class DatasetWine(Dataset):
    def __init__(self):
        #data loading
        xy=np.loadtxt(url, delimiter=',', dtype='float32', skiprows=1)
        self.x=torch.from_numpy(xy[:,1:])
        self.y=torch.from_numpy(xy[:,[0]])
        self.n_sample= xy.shape[0]
        
    def __getitem__(self,index):
        return self.x[index] , self.y[index]
    
    def __len__(self):
        return self.n_sample

dataset=DatasetWine()        

In [4]:
first_data= dataset[0]
feature, target =first_data
print(feature, target)

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [5]:
dataloader= DataLoader(dataset=dataset,batch_size=10,shuffle=True)

In [6]:
dataiter= iter(dataloader)
data= next(dataiter)
features, labels = data
print(features, labels)

tensor([[1.3710e+01, 1.8600e+00, 2.3600e+00, 1.6600e+01, 1.0100e+02, 2.6100e+00,
         2.8800e+00, 2.7000e-01, 1.6900e+00, 3.8000e+00, 1.1100e+00, 4.0000e+00,
         1.0350e+03],
        [1.3580e+01, 2.5800e+00, 2.6900e+00, 2.4500e+01, 1.0500e+02, 1.5500e+00,
         8.4000e-01, 3.9000e-01, 1.5400e+00, 8.6600e+00, 7.4000e-01, 1.8000e+00,
         7.5000e+02],
        [1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03],
        [1.2080e+01, 1.3900e+00, 2.5000e+00, 2.2500e+01, 8.4000e+01, 2.5600e+00,
         2.2900e+00, 4.3000e-01, 1.0400e+00, 2.9000e+00, 9.3000e-01, 3.1900e+00,
         3.8500e+02],
        [1.3830e+01, 1.6500e+00, 2.6000e+00, 1.7200e+01, 9.4000e+01, 2.4500e+00,
         2.9900e+00, 2.2000e-01, 2.2900e+00, 5.6000e+00, 1.2400e+00, 3.3700e+00,
         1.2650e+03],
        [1.2080e+01, 2.0800e+00, 1.7000e+00, 1.7500e+01, 9.7000e+01, 2.2300e+00,

#### Insert a transform method to the data

In [7]:
class DatasetWine_2(Dataset):
    def __init__(self, transform=None): # transform is fix to None by default
        #data loading
        xy=np.loadtxt(url, delimiter=',', dtype='float32', skiprows=1)
        self.x=xy[:,1:]
        self.y=xy[:,[0]] # we gonna do the transformation to Tensor with a transform
        self.n_sample= xy.shape[0]
        self.transform= transform # we initialize the transform with the function when we call DatasetWine_2 
        
        
    def __getitem__(self,index):
        
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample= self.transform(sample)# we apply the transform on the data
            
        return sample
    
    def __len__(self):
        return self.n_sample
    
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
    
dataset=DatasetWine_2(transform= ToTensor())
first_data=  dataset[2]
feature, label= first_data
print(feature, label)

tensor([1.3160e+01, 2.3600e+00, 2.6700e+00, 1.8600e+01, 1.0100e+02, 2.8000e+00,
        3.2400e+00, 3.0000e-01, 2.8100e+00, 5.6800e+00, 1.0300e+00, 3.1700e+00,
        1.1850e+03]) tensor([1.])


In [8]:
class MulTransf:
    def __init__(self, factor):
        self.factor=factor
    def __call__(self,sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target
dataset=DatasetWine_2(transform= MulTransf(2))
first_data=  dataset[2]
feature, label= first_data
print(feature, label) # here we have numpy array at the end because we don't apply transform to tensor

[2.632e+01 4.720e+00 5.340e+00 3.720e+01 2.020e+02 5.600e+00 6.480e+00
 6.000e-01 5.620e+00 1.136e+01 2.060e+00 6.340e+00 2.370e+03] [1.]


#### Composed Transforms

In [9]:
composed= torchvision.transforms.Compose([ToTensor(), MulTransf(3)])
dataset=DatasetWine_2(transform= composed)
first_data=  dataset[2]
feature, label= first_data
print(feature, label)

tensor([3.9480e+01, 7.0800e+00, 8.0100e+00, 5.5800e+01, 3.0300e+02, 8.4000e+00,
        9.7200e+00, 9.0000e-01, 8.4300e+00, 1.7040e+01, 3.0900e+00, 9.5100e+00,
        3.5550e+03]) tensor([1.])
